In [1]:
import numpy as np
# Example numpy file for single-input ONNX
calib_data = np.random.randn(1, 1, 64, 256, 256)
calib_data = calib_data.astype(np.float32)
np.save("calib_data.npy", calib_data)

# Example numpy file for single/multi-input ONNX
# Dict key should match the input names of ONNX
# calib_data = {
#     "input_name": np.random.randn(*shape),
#     "input_name2": np.random.randn(*shape2),
# }
# np.savez("/workspace/calib_data.npz", calib_data)

import sys

sys.path.insert(0, './TensorRT-Model-Optimizer')

import modelopt.onnx.quantization as moq
import numpy as np

calibration_data_path = 'calib_data.npy'
# onnx_path = "vit_base_patch16_224.onnx"
#
calibration_data = np.load(calibration_data_path)

moq.quantize(
    onnx_path="onnx_models/fast_unet.onnx",
    calibration_data=calibration_data,
    output_path="onnx_models/quant_fast_unet.onnx",
    quantize_mode="int8",
    # high_precision_dtype="float16",
)

INFO:root:Model onnx_models/fast_unet.onnx with opset_version 18 is loaded.
INFO:root:Quantization Mode: int8
INFO:root:Quantizable op types in the model: ['Conv']
INFO:root:Building non-residual Add input map ...
INFO:root:Searching for hard-coded patterns like MHA, LayerNorm, etc. to avoid quantization.
INFO:root:Building KGEN/CASK targeted partitions ...
INFO:root:Classifying the partition nodes ...
INFO:root:Total number of nodes: 91


100%|██████████| 27/27 [00:08<00:00,  3.33it/s]


Finding optimal threshold for each tensor using 'entropy' algorithm ...
Number of tensors : 27
Number of histogram bins : 128 (The number may increase depends on the data it collects)
Number of quantized bins : 128


INFO:root:Deleting QDQ nodes from marked inputs to make certain operations fusible ...
INFO:root:Total number of quantized nodes: 33
INFO:root:Quantized type counts: {'Conv': 27, 'Concat': 6}
INFO:root:Quantized onnx model is saved as onnx_models/quant_fast_unet.onnx


next_node.op_type: Conv
before: (14, 32, 1, 1, 1)
after: (1, 1, 1, 32, 14)
np_y_scale: (14,)
next_node.op_type: Conv
before: (320, 640, 3, 3, 3)
after: (3, 3, 3, 640, 320)
np_y_scale: (320,)
next_node.op_type: Conv
before: (320, 320, 3, 3, 3)
after: (3, 3, 3, 320, 320)
np_y_scale: (320,)
next_node.op_type: Conv
before: (320, 640, 3, 3, 3)
after: (3, 3, 3, 640, 320)
np_y_scale: (320,)
next_node.op_type: Conv
before: (320, 320, 3, 3, 3)
after: (3, 3, 3, 320, 320)
np_y_scale: (320,)
next_node.op_type: Conv
before: (256, 512, 3, 3, 3)
after: (3, 3, 3, 512, 256)
np_y_scale: (256,)
next_node.op_type: Conv
before: (256, 256, 3, 3, 3)
after: (3, 3, 3, 256, 256)
np_y_scale: (256,)
next_node.op_type: Conv
before: (128, 256, 3, 3, 3)
after: (3, 3, 3, 256, 128)
np_y_scale: (128,)
next_node.op_type: Conv
before: (128, 128, 3, 3, 3)
after: (3, 3, 3, 128, 128)
np_y_scale: (128,)
next_node.op_type: Conv
before: (64, 128, 3, 3, 3)
after: (3, 3, 3, 128, 64)
np_y_scale: (64,)
next_node.op_type: Conv
befo

In [2]:
!/usr/src/tensorrt/bin/trtexec --onnx=onnx_models/quant_fast_unet.onnx --saveEngine=onnx_models/quant_fast_unet.engine --best

&&&& RUNNING TensorRT.trtexec [TensorRT v100800] [b43] # /usr/src/tensorrt/bin/trtexec --onnx=onnx_models/quant_fast_unet.onnx --saveEngine=onnx_models/quant_fast_unet.engine --best
[02/28/2025-06:18:14] [I] === Model Options ===
[02/28/2025-06:18:14] [I] Format: ONNX
[02/28/2025-06:18:14] [I] Model: onnx_models/quant_fast_unet.onnx
[02/28/2025-06:18:14] [I] Output:
[02/28/2025-06:18:14] [I] === Build Options ===
[02/28/2025-06:18:14] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default, tacticSharedMem: default
[02/28/2025-06:18:14] [I] avgTiming: 8
[02/28/2025-06:18:14] [I] Precision: FP32+FP16+BF16+INT8
[02/28/2025-06:18:14] [I] LayerPrecisions: 
[02/28/2025-06:18:14] [I] Layer Device Types: 
[02/28/2025-06:18:14] [I] Calibration: Dynamic
[02/28/2025-06:18:14] [I] Refit: Disabled
[02/28/2025-06:18:14] [I] Strip weights: Disabled
[02/28/2025-06:18:14] [I] Version Compatible: Disabled
[02/28/2025-06:18:14] [I] ONNX Plugin InstanceNorm: 

In [5]:
!/usr/src/tensorrt/bin/trtexec --onnx=onnx_models/quant_fast_unet.onnx --saveEngine=onnx_models/quant_fast_unet_fp16.engine --fp16

&&&& RUNNING TensorRT.trtexec [TensorRT v100800] [b43] # /usr/src/tensorrt/bin/trtexec --onnx=onnx_models/quant_fast_unet.onnx --saveEngine=onnx_models/quant_fast_unet_fp16.engine --fp16
[02/28/2025-03:40:51] [I] === Model Options ===
[02/28/2025-03:40:51] [I] Format: ONNX
[02/28/2025-03:40:51] [I] Model: onnx_models/quant_fast_unet.onnx
[02/28/2025-03:40:51] [I] Output:
[02/28/2025-03:40:51] [I] === Build Options ===
[02/28/2025-03:40:51] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default, tacticSharedMem: default
[02/28/2025-03:40:51] [I] avgTiming: 8
[02/28/2025-03:40:51] [I] Precision: FP32+FP16
[02/28/2025-03:40:51] [I] LayerPrecisions: 
[02/28/2025-03:40:51] [I] Layer Device Types: 
[02/28/2025-03:40:51] [I] Calibration: 
[02/28/2025-03:40:51] [I] Refit: Disabled
[02/28/2025-03:40:51] [I] Strip weights: Disabled
[02/28/2025-03:40:51] [I] Version Compatible: Disabled
[02/28/2025-03:40:51] [I] ONNX Plugin InstanceNorm: Disabled
[02

In [1]:
!sudo apt-get -y install tensorrt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libnvinfer-bin libnvinfer-dev libnvinfer-dispatch-dev libnvinfer-dispatch10
  libnvinfer-headers-dev libnvinfer-headers-plugin-dev libnvinfer-lean-dev
  libnvinfer-lean10 libnvinfer-plugin-dev libnvinfer-plugin10
  libnvinfer-samples libnvinfer-vc-plugin-dev libnvinfer-vc-plugin10
  libnvinfer10 libnvonnxparsers-dev libnvonnxparsers10 python3-libnvinfer
  python3-libnvinfer-dev python3-libnvinfer-dispatch python3-libnvinfer-lean
The following NEW packages will be installed:
  libnvinfer-bin libnvinfer-dev libnvinfer-dispatch-dev libnvinfer-dispatch10
  libnvinfer-headers-dev libnvinfer-headers-plugin-dev libnvinfer-lean-dev
  libnvinfer-lean10 libnvinfer-plugin-dev libnvinfer-plugin10
  libnvinfer-samples libnvinfer-vc-plugin-dev libnvinfer-vc-plugin10
  libnvinfer10 libnvonnxparsers-dev libnvonnxparsers10 python3-libnvinfer
  python3-l

In [5]:
!sudo apt-get -y install tensorrt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package tensorrt is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Version '10.0.1' for 'tensorrt' was not found


In [ ]:
!sudo apt-get -y install cudnn9-cuda-12

In [6]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.6 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... one
  Getting requirements to build wheeldone
doneproject.toml) ... 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 KB 7.4 MB/s eta 0:00:00
done
  Created wheel for pycuda: filename=pycuda-2025.1-cp310-cp310-linux_x86_64.whl size=660661 sha256=46f5dd78dc33628a5975d331b9e4b6962fabd563de5e117ce14c5e45f6d148f3
  Stored in directory: /root/.cache/pip/wheels/65/53/5f/f5f184c26b7cc503acb77f3456531a6e1fac0ce30c774b9d82
Successfully built pycuda


In [1]:
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

f = open("onnx_models/quant_fast_unet_fp16.engine", "rb")
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING)) 

engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

In [2]:
import numpy as np
BATCH_SIZE = 1
target_dtype = np.float32
# need to set input and output precisions to FP16 to fully enable it
output = np.empty([1, 14, 64, 256, 256], dtype = target_dtype) 

input_batch = np.array([1, 1, 64, 256, 256], dtype=target_dtype)

# allocate device memory
d_input = cuda.mem_alloc(1 * input_batch.nbytes)
d_output = cuda.mem_alloc(1 * output.nbytes)

tensor_names = [engine.get_tensor_name(i) for i in range(engine.num_io_tensors)]
assert(len(tensor_names) == 2)

context.set_tensor_address(tensor_names[0], int(d_input))
context.set_tensor_address(tensor_names[1], int(d_output))

bindings = [int(d_input), int(d_output)]

stream = cuda.Stream()

In [3]:
def predict(batch): # result gets copied into output
    # transfer input data to device
    cuda.memcpy_htod_async(d_input, batch, stream)
    # execute model
    # start=time.time()
    # for i in range(100):
    context.execute_async_v3(stream.handle)
    # print(time.time() - start)
    # transfer predictions back
    cuda.memcpy_dtoh_async(output, d_output, stream)
    # syncronize threads
    # stream.synchronize()
    
    return output

In [4]:
pred = predict(input_batch)

[02/28/2025-06:15:20] [TRT] [E] IExecutionContext::enqueueV3: Error Code 1: Cuda Runtime (an illegal memory access was encountered)


LogicError: cuMemcpyDtoHAsync failed: an illegal memory access was encountered